In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np
import os

#basic cnn
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


train_datagen = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1/255)

training_set = train_datagen.flow_from_directory('G:/Downloads/TomatoLeafDiseasemain/DiseasePrediction/TomatoLeafDisease/Dataset/train', # relative path from working directoy
                                                 target_size = (128, 128),
                                                 batch_size = 6, class_mode = 'categorical')
valid_set = test_datagen.flow_from_directory('G:/Downloads/TomatoLeafDiseasemain/DiseasePrediction/TomatoLeafDisease/Dataset/val', # relative path from working directoy
                                             target_size = (128, 128), 
                                        batch_size = 3, class_mode = 'categorical')

labels = (training_set.class_indices)
print(labels)


Found 143 images belonging to 11 classes.
Found 52 images belonging to 10 classes.
{'.ipynb_checkpoints': 0, 'Bacterial_spot': 1, 'Early_blight': 2, 'Healthy': 3, 'Late_blight': 4, 'Leaf_Mold': 5, 'Mosaic_virus': 6, 'Septoria_leaf_spot': 7, 'Target_Spot': 8, 'Two-spotted_spider_mite': 9, 'Yellow_Leaf_Curl_Virus': 10}


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Embedding,Flatten

from sklearn.model_selection import train_test_split

X_train = np.zeros((180568, 80, 1))
y_train = np.zeros((180568, 80))
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_samples = 1000

X = np.random.uniform(0,1, (n_samples, n_timesteps, n_features))
y = pd.get_dummies(np.random.randint(0,n_outputs, n_samples)).values

model = tf.keras.models.Sequential([
tf.keras.layers.Conv1D(input_shape=(n_timesteps, n_features), kernel_size=3, filters=16),
tf.keras.layers.GlobalAveragePooling1D(), # also GlobalMaxPooling1D() is ok
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(n_outputs, activation='softmax')
])

model.compile('adam', 'categorical_crossentropy')
model.fit(X,y, epochs=50,verbose=1)
model.summary()

Train on 1000 samples
Epoch 1/50
1000/1000 [==============================] - 2s 2ms/sample - loss: 4.3822
Epoch 2/50
1000/1000 [==============================] - 0s 81us/sample - loss: 4.3706
Epoch 3/50
1000/1000 [==============================] - 0s 81us/sample - loss: 4.3597
Epoch 4/50
1000/1000 [==============================] - 0s 81us/sample - loss: 4.3536
Epoch 5/50
1000/1000 [==============================] - 0s 83us/sample - loss: 4.3493
Epoch 6/50
1000/1000 [==============================] - 0s 83us/sample - loss: 4.3503
Epoch 7/50
1000/1000 [==============================] - 0s 81us/sample - loss: 4.3456
Epoch 8/50
1000/1000 [==============================] - 0s 82us/sample - loss: 4.3501
Epoch 9/50
1000/1000 [==============================] - 0s 83us/sample - loss: 4.3497
Epoch 10/50
1000/1000 [==============================] - 0s 83us/sample - loss: 4.3469
Epoch 11/50
1000/1000 [==============================] - 0s 82us/sample - loss: 4.3441
Epoch 12/50
1000/1000 [========

In [ ]:
#Import necessary libraries
from flask import Flask, render_template, request

import numpy as np
import os

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

filepath = 'G:/Downloads/TomatoLeafDiseasemain/DiseasePrediction/model.h5'
model = load_model(filepath)
print(model)

print("Model Loaded Successfully")

def pred_tomato_dieas(tomato_plant):
  test_image = load_img(tomato_plant, target_size = (128, 128)) # load image 
  print("@@ Got Image for prediction")
  
  test_image = img_to_array(test_image)/255 # convert image to np array and normalize
  test_image = np.expand_dims(test_image, axis = 0) # change dimention 3D to 4D
  
  result = model.predict(test_image) # predict diseased palnt or not
  print('@@ Raw result = ', result)
  
  pred = np.argmax(result, axis=1)
  print(pred)
  if pred==0:
      return "Tomato - Bacteria Spot Disease", 'Tomato-Bacteria Spot.html'
       
  elif pred==1:
      return "Tomato - Early Blight Disease", 'Tomato-Early_Blight.html'
        
  elif pred==2:
      return "Tomato - Healthy and Fresh", 'Tomato-Healthy.html'
        
  elif pred==3:
      return "Tomato - Late Blight Disease", 'Tomato - Late_blight.html'
       
  elif pred==4:
      return "Tomato - Leaf Mold Disease", 'Tomato - Leaf_Mold.html'
        
  elif pred==5:
      return "Tomato - Septoria Leaf Spot Disease", 'Tomato - Septoria_leaf_spot.html'
        
  elif pred==6:
      return "Tomato - Target Spot Disease", 'Tomato - Target_Spot.html'
        
  elif pred==7:
      return "Tomato - Tomoato Yellow Leaf Curl Virus Disease", 'Tomato - Tomato_Yellow_Leaf_Curl_Virus.html'
  elif pred==8:
      return "Tomato - Tomato Mosaic Virus Disease", 'Tomato - Tomato_mosaic_virus.html'
        
  elif pred==9:
      return "Tomato - Two Spotted Spider Mite Disease", 'Tomato - Two-spotted_spider_mite.html'

    

# Create flask instance
app = Flask(__name__)

# render index.html page
@app.route("/", methods=['GET', 'POST'])
def home():
        return render_template('index.html')
    
 
# get input image from client then predict class and render respective .html page for solution
@app.route("/predict", methods = ['GET','POST'])
def predict():
     if request.method == 'POST':
        file = request.files['image'] # fet input
        filename = file.filename        
        print("@@ Input posted = ", filename)
        
        file_path = os.path.join('G:/Downloads/TomatoLeafDiseasemain/DiseasePrediction/static/upload/', filename)
        file.save(file_path)

        print("@@ Predicting class......")
        pred, output_page = pred_tomato_dieas(tomato_plant=file_path)
              
        return render_template(output_page, pred_output = pred, user_image = file_path)
    
# For local system & cloud
if __name__ == "__main__":
    app.run(threaded=False,port=8080) 
    
    


Model Loaded Successfully
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jun/2022 23:51:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:51:09] "GET /static/images/Background.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:51:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Jun/2022 23:51:58] "GET / HTTP/1.1" 200 -


@@ Input posted =  Early_blight (1).JPG
@@ Predicting class......
@@ Got Image for prediction


c:\users\ankit sonkar\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
127.0.0.1 - - [16/Jun/2022 23:51:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:51:59] "GET /static/images/Background.jpg HTTP/1.1" 304 -


@@ Raw result =  [[7.78737843e-01 9.97640014e-01 5.67346811e-04 9.28934574e-01
  9.68013108e-01 7.91177630e-01 1.11192778e-04 7.00541198e-01
  3.82989645e-04 1.02954835e-01]]
[1]


127.0.0.1 - - [16/Jun/2022 23:52:06] "POST /predict HTTP/1.1" 200 -


@@ Input posted =  Yellow_Leaf_Curl_Virus (1).JPG
@@ Predicting class......
@@ Got Image for prediction
@@ Raw result =  [[9.9378240e-01 2.7540326e-04 8.3317161e-03 5.5426955e-03 2.6724023e-01
  9.9826288e-01 3.5113096e-04 9.9999404e-01 2.6906684e-01 4.3492830e-01]]
[7]


127.0.0.1 - - [16/Jun/2022 23:52:06] "GET /static/images/Tomato___Tomato_Yellow_Leaf_Curl_Virus.JPG HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:52:07] "GET /static/images/Back.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:52:17] "POST /predict HTTP/1.1" 200 -


@@ Input posted =  Leaf_Mold (1).JPG
@@ Predicting class......
@@ Got Image for prediction
@@ Raw result =  [[7.0745718e-01 9.7948825e-01 9.9811333e-01 9.9507785e-01 9.7452676e-01
  9.7846854e-01 4.3002158e-02 1.7082691e-04 1.6368628e-03 3.3488572e-03]]
[2]


127.0.0.1 - - [16/Jun/2022 23:52:17] "GET /static/images/Tomato___healthy%20.JPG HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2022 23:52:18] "GET /static/images/Back.jpg HTTP/1.1" 304 -
